---
# **LA LOI DU CROUS :**
## *Étude des déterminations du prix des résidences universitaires Crous et de leurs répartitions dans les départements*
---

# Introduction:

350 000 étudiants habitent dans une résidence des centres régionaux des œuvres universitaires et scolaires (Crous), soit seulement 10% des 2 millions d'étudiants vivant hors de leurs domiciles, selon un  rapport sur le logement et la précarité des étudiants, des apprentis et des jeunes actifs de l'Assemblée nationale datant de janvier 2022 [1]. Selon ce même rapport, le logement est le premier poste de dépense des étudiants soit 60% de leurs revenus. Il est donc primordiale de donner accès à un logement abordable aux étudiants qui sont de plus en plus précaires de manière équitable selon les départements où ils se trouvent. 

Partant de ces constats, nous nous sommes posés plusieurs questions : le loyer est-il le même pour un logement situé dans un marché immobilier en tension comme Paris que dans un marché qu'il l'est moins comme Dijon ? Ou au contraire, est-ce que les prix sont globalement uniformes ? De même, est-ce que la concentration importante d'étudiants scolarisés à proximité de la résidence influe le prix vers la hausse? 

De ce fait, nous chercherons donc à répondre à la problématique suivante : 

***Les loyers des résidences des Crous suivent-ils la loi de l'offre et de la demande ?***

**Modèle :**

Ainsi, nous proposons d'étudier dans ce rapport, d'une part la répartition des 795 résidences des Crous sur le territoire; et d'autre part, les déterminants du prix du loyer de celles-ci pour pouvoir montrer d'éventuelles variations entre les départements. Pour expliquer le loyer nous utiliserons un modèle d'offre et de demande économique élémentaire. Dans celui-ci, la résidence offre des biens de surfaces variables q à un prix p en mètre carré, et fait face à une concurrence représenté par le montant des loyers dans le parc privé. La demande sera représentée par l'effectif des étudiants scolarisés dans une école se situant dans un rayon de 20km de la résidence. Nous faisons l'hypothèse que les résidences proposent à peu près les mêmes caractéristiques entre elles et que les étudiants sont des price-takers. 

Nous réaliserons ainsi des régressions linéaires classiques : 

\begin{equation}
\hat{Y}_i = \hat{\beta}_0 + \hat{\beta}_1 X_i + \hat{\epsilon}_i
\end{equation}

**PLAN:**

1. Dans une première partie, nous réccupérons une base de données avec les loyers (maximum, minimum et moyen), les surfaces (idem), les coordonnées géographiques, les loyers privés dans la ville de la résidence et l'effectif des étudiants dans un périmètre de 20 kilomètres. 

2. Dans une deuxième partie, nous analyserons d'une part la répartition des résidences en France et, d'autre part, nous réaliserons des statistiques descriptives sur les différentes variables de notre base de données. 

3. Dernièrement, nous utiliserons des modèles de régressions linéaires pour répondre à notre problématique. 


## Les données utilisées 

Nous nous servirons de plusieurs bases de données : 

1. **Ensemble des logements proposés aux étudiants par le réseau des CROUS :** 

Données mises à disposition par le Centre National des Œuvres Universitaires et Scolaires (CROUS). Elles ont été publiées en 2017 et sont mises à jour assez régulièrement. Le problème principal de cette base de données est la récupération des loyers et des surfaces qui sont stockés dans une colonne nommée "infos" (s'il y en a). De plus, chaque colonne a été remplie par les résidences. La qualité et la structure des informations sont donc très variables. Nous avons donc mis en place des techniques de traitements textuels ou de natural language processing (NPL).

https://www.data.gouv.fr/fr/datasets/ensemble-des-logements-proposes-aux-etudiants-par-le-reseau-des-crous/

2. **"Carte des loyers" - Indicateurs de loyers d'annonce par commune en 2018**

Données obtenues par une collaboration entre la Direction Générale de l’Aménagement, du Logement et de la Nature (DGALN), une équipe de recherche en économie d’Agrosup Dijon, de l’Institut national de la recherche en agronomique (INRAE), de SeLoger, de leboncoin et de PAP. Ces équipes ont créé différents indicateurs par communes à partir de 9 millions d'annonces. Les données datent de 2018 et nous nous servirons uniquement de données pour les appartements.

https://www.data.gouv.fr/fr/datasets/carte-des-loyers-indicateurs-de-loyers-dannonce-par-commune-en-2018/


3. **Statistiques sur les effectifs d'étudiants inscrits par établissement public sous tutelle du ministère en charge de l'Enseignement supérieur (avec doubles inscriptions CPGE)**

"Ce jeu de données propose un ensemble de statistiques sur les effectifs d'étudiants inscrits de 2006-07 à 2021-22 par établissement public sous tutelle du ministère en charge de l'Enseignement supérieur français : universités, universités de technologie, grands établissements, COMUE, écoles normales supérieures, écoles centrales, INSA, autres écoles d'ingénieurs..."

https://www.data.gouv.fr/fr/datasets/statistiques-sur-les-effectifs-detudiants-inscrits-par-etablissement-public-sous-tutelle-du-ministere-en-charge-de-lenseignement-superieur-avec-doubles-inscriptions-cpge/

4. **Contours Communes France Administrative "Format Admin-Express" avec arrondissements**

Jeu de données de l'IGN regroupant les contours communales avec arrondissements.

https://www.data.gouv.fr/fr/datasets/contours-communes-france-administrative-format-admin-express-avec-arrondissements/

5. **Contours des départements français issus d'OpenStreetMap**

Jeu de données d'OpenStreetMap regroupant les contours géographiques des départements. 

https://www.data.gouv.fr/fr/datasets/contours-des-departements-francais-issus-d-openstreetmap/





# Librairies nécessaires

In [1]:
%%capture 
#hide output 
!pip install geoplot
!pip install contextily
!pip install altair

In [2]:
import pandas as pd 
import geopandas as gpd 
import nltk
nltk.download('punkt')
import contextily as ctx
import geoplot as gplt
import geoplot.crs as gcrs
import matplotlib.pyplot as plt
import folium
import numpy as np
import seaborn as sns
import altair as alt 
import statsmodels.stats.api as sms
from statsmodels.stats.diagnostic import het_breuschpagan

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
ERROR 1: PROJ: proj_create_from_database: Open of /opt/mamba/share/proj failed


# Partie I : Obtention du dataframe 

L'objectif de cette partie est de créer une base de données contenant pour chaque résidence : 
- Le prix du loyer (maximum, minimum et moyen);
- La surface (idem);
- Les coordonnées géographiques compatibles avec Géopanda (nécessaires pour réccupèrer le nombre d'étudiants et les cartes);
- L'indicateur du loyer privé maximum d'un appartement dans la ville de la résidence;
- Le nombre d'étudiants scolarisés dans un rayon de 20 km autour de la résidence hors CPGE

Le principal problème de cette base de données est le fait que d'une part, il n'existe pas de colonne contenant exlicitement le loyer et les surfaces, et d'autre part, chaque résidence a rempli à sa manière les informations sur les loyers et les surfaces (s'ils ont rempli). Nous avons donc mis en place des fonctions de Natural Language Processing (NLP) pour pouvoir les réccupèrer (pour plus d'informations voire le document : functions_for_data_cleaning.py) 

## I.1. Récupération des loyers, des surfaces et de la localisation des résidences

In [3]:
"""
Importation des données.
"""
df = pd.read_csv("https://data.enseignementsup-recherche.gouv.fr//explore/dataset/fr_crous_logement_france_entiere/download?format=csv&timezone=Europe/Berlin&use_labels_for_header=false", 
                 sep =';')

In [4]:
"""
Réduction de notre base de données aux 5 colonnes qui nous intéressent (nom de la résidence ou "title";
la colonne comportant toutes les infos sur les surfaces et les loyers ; l'adresse; les coordonnées géographiques; 
et les noms des régions.
"""
df = df[["title", "infos", "address" ,"geocalisation", "regions"]]

In [5]:
from functions_for_data_cleaning import * #document .py comportant nos fonctions nécessaires au nettoyage

df = get_loyer(df) #récuppération des loyers à partir de la colonne "infos"
df = get_surface(df) #récuppération des surfaces à partir de la colonne "infos"
df = get_localisation(df) #récuppération des coordonnées géographiques à partir de la colonne "geocalisation"

[nltk_data] Downloading package punkt to /home/onyxia/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## I.2. Récupération des loyers privés des communes (avec la géolocalisation des communes)

Le but de cette partie est de joindre la base précédemment obtenue avec la base des indicateurs des loyers privés maximum pour un appartement et par commune (var : upr.lPm2)

In [6]:
df_private = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/8fac6fb7-cd07-4747-8e0b-b101c476f0da", encoding= 'unicode_escape', sep = ";")

On ne garde que le code INSEE, le nom de la ville et le loyer supérieur par m^2


In [7]:
df_private = df_private[["INSEE", "LIBGEO","upr.IPm2"]]

On renomme la colonne INSEE en codgeo pour pouvoir la merger avec la carte des contours des villes.

In [8]:
df_private.rename(columns = {'INSEE':'codgeo',"upr.IPm2" : "Loyer_prive_m2" }, inplace = True)

In [9]:
"""
On rajoute un 0 devant le code INSEE quand il ne fait que quatre chiffres (pour uniformiser avec l'autre base).
"""
df_private["codgeo"] = df_private["codgeo"].apply(lambda x : "0" + x if len(x) == 4 else x)

"""
On transforme en float le loyer privé 
"""
df_private["Loyer_prive_m2"] = df_private["Loyer_prive_m2"].apply(lambda x : x.replace(',','.')) 
df_private["Loyer_prive_m2"] = df_private["Loyer_prive_m2"].astype(float)

In [10]:
df_private = df_private.set_index('codgeo') #on réindex le dataframe avec le code INSEE pour merger après. 

On récupére les contours géographique des communes pour pouvoir faire la jointure avec notre base.(prends du temps)

In [11]:
gdf_communes_boundaries = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/e9391593-fa95-4153-aabe-87ca84d197e9")

On ne garde que le code INSEE et les données polygoniales. 

In [12]:
gdf_communes_boundaries.rename(columns = {'INSEE_COM':'codgeo'}, inplace = True)
gdf_communes_boundaries = gdf_communes_boundaries[["codgeo", "geometry"]]
gdf_communes_boundaries = gdf_communes_boundaries.set_index('codgeo')


On concatène les deux dataframes

In [13]:
gdf_private = pd.concat([df_private,gdf_communes_boundaries], axis = 1, join = "inner")

## I.3. Jointures spatiales

Maintenant que nous avons une base des loyers privés avec les coordonnées des communes nous allons pouvoir réaliser une jointure spatiale avec la base des résidences.

In [ ]:

#gdf_private['geometry'] = gdf_private['geometry'].apply(wkt.loads) #on transforme en format wkt (sinon ça ne marche pas)
gdf_private = gpd.GeoDataFrame(gdf_private)  #on transforme en geodataframe 
df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within 
#i.e si le point est dans le polynôme. 

/opt/mamba/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3373: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):
/tmp/ipykernel_727/2580680376.py:3: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  df = gpd.sjoin(df, gdf_private, how='left', op='within') #on réalise la jointure à gauche avec within


In [ ]:
del df['index_right'] #on supprime l'index right pour la prochaine jointure

## I.4 Ajout des départements 

Pour pouvoir avoir des comparaisons par départements nous allons récupérer les départements et les joindre avec nos bases grâce à une jointure spatiale. 

In [ ]:
gdf_dep = gpd.read_file("https://www.data.gouv.fr/fr/datasets/r/eb36371a-761d-44a8-93ec-3d728bec17ce")
gdf_dep.rename(columns = {'nom':'departement'}, inplace = True)
gdf_dep = gdf_dep[["departement", "geometry", "code_insee"]]

In [ ]:
df = gpd.sjoin(df, gdf_dep, how='left', op='within')
del df['index_right'] #on supprime l'index right pour la prochaine jointure

In [ ]:
gdf_dep

## I.5. Association de résidences à une école 

Import de notre base de données des écoles pour pouvoir récupérer le nombre d'étudiants scolarisés dans un rayon de 20km.

In [ ]:
df_schools = pd.read_csv("https://www.data.gouv.fr/fr/datasets/r/0c713161-26fb-415e-ac1d-8769125f338d", sep = ";")

In [ ]:
df_schools

In [ ]:
df_schools = df_schools[df_schools["annee_universitaire"] == "2021-22" ] #on ne prend que l'année 2021-22
df_schools = df_schools[["etablissement_lib", "effectif_sans_cpge", "etablissement_code_commune", "etablissement_commune",  "etablissement_departement"]]
df_schools.rename(columns = {'etablissement_code_commune':'codgeo'}, inplace = True) #On renomme la colonne pour la jointure

On enlève les valeurs manquantes (il manque des coordonnées pour certains établissments).

In [ ]:
index_with_nan = df_schools.index[df_schools.isnull().any(axis=1)]
df_schools.drop(index_with_nan,0, inplace=True) 
df_schools.set_index('codgeo', inplace = True) #on recommence l'index

Jointure avec les coordonnées géographiques des communes pour pouvoir trouver dans un rayon de 20km les écoles autour des résidences.

In [ ]:
df_schools = df_schools.join(gdf_communes_boundaries)
df_schools = gpd.GeoDataFrame(df_schools)
df_schools.crs = "epsg:4326"

On applique la fonction get_nb_student disponible dans le fichier function_for_data_cleaning.py qui permet d'obtenir le nombre d'étudiants dans un rayon de 20km autour de nos résidences.

In [ ]:
df.crs = "epsg:4326" #on définit la projection de notre dataframe
df = get_nb_student(df, df_schools, 20)

## I.6. Obtention de la base de données finale

In [ ]:
df 

# Part II : Visualisation 

## II.0 Construction d'indicateurs 

Construction d'indicateurs du prix des loyers par m2 ('Loyer_crous_prix_par_m2_surf_min' en référence aux studios et 'Loyer_crous_prix_par_m2_surf_max' en référence aux (grands) studios/colocation)

In [ ]:
df['Loyer_crous_prix_par_m2_surf_min'] = df['Min Loyer']/df['Min Surface'] 
df['Loyer_crous_prix_par_m2_surf_max'] = df['Max Loyer']/df['Max Surface'] 

Construction d'un indicateur du nombre de Crous par nombre d'étudiants par département qui sera ensuite affiché sur une carte. Cela va créer un ratio en pourcentage.  

In [ ]:
Nb_residence_par_dep = df.reset_index().groupby('departement')['index'].nunique() #on reindex le dataframe
#de 0 à n-1 puis on compte le nombre d'index unique par département pour avoir le nombre de résidences par départements
Nb_residence_par_dep = pd.DataFrame(Nb_residence_par_dep) #on transforme en dataframe 
nb_eleve_dep = df_schools.groupby('etablissement_departement').sum()['effectif_sans_cpge'] #on somme par département le nb
#le nombre d'étudiants
nb_eleve_dep = pd.DataFrame(nb_eleve_dep) #on transforme en dataframe pour la jointure
nb_residence_par_eleves = Nb_residence_par_dep.join(nb_eleve_dep, how = 'inner') #on joint nos deux dataframe
nb_residence_par_eleves = nb_residence_par_eleves.assign(Nb_by_students=lambda x: (x["index"]/x["effectif_sans_cpge"])*100)
#On crée notre ratio du nb de résidences/élèves/dép 
nb_residence_par_eleves = nb_residence_par_eleves.join(gdf_dep.set_index("departement"))
#on rajoute les coordonnées géographiques des départements (pour les cartes) 
nb_residence_par_eleves.reset_index(inplace = True)
#on reset index 
del nb_residence_par_eleves["geometry"]
#on supprime la colonne geometry par simplicité par la suite 
nb_residence_par_eleves

Dataframe avec les départements avec le plus d'étudiants dans l'ordre décroissant 

In [ ]:
nb_eleve_dep_top_10 = nb_eleve_dep.sort_values(by='effectif_sans_cpge', ascending=False) 
nb_eleve_dep_top_10

Dataframe avec les départements avec le plus de résidences Crous dans l'ordre décroissant 

In [ ]:
nb_res_dep_top_10 = Nb_residence_par_dep.sort_values(by='index', ascending=False) 
nb_res_dep_top_10

Construction d'un ratio pour enlever les départements avec un nombre de valeurs importantes (surfaces ou loyers) trop important 

In [ ]:
df_ratio_min = df
df_ratio_min['prix_par_m2_surf_min_null'] = df.Loyer_crous_prix_par_m2_surf_min.isnull() # On compte le nombre de valeurs manquantes par ligne
df_ratio_min['prix_par_m2_surf_min_null'] = df_ratio_min['prix_par_m2_surf_min_null'].astype(int) # on transforme le booléen en valeur 0 ou 1 (1 voulant dire que la valeur est manquante)
df_ratio_min = df_ratio_min.groupby(['departement'], as_index=False)['prix_par_m2_surf_min_null'].sum() # On additionne par département le nombre de résidences avec des valeurs manquantes
df_ratio_min = pd.merge(df_ratio_min, Nb_residence_par_dep, how="inner", on=[Nb_residence_par_dep.index, "departement"])
df_ratio_min = df_ratio_min.rename(columns={'index': 'Nb_res_dep'})
df_ratio_min['ratio'] = df_ratio_min['prix_par_m2_surf_min_null']/df_ratio_min['Nb_res_dep']
df_ratio_min = df_ratio_min[df_ratio_min['ratio']<0.5]    # On enlève les départements dont plus de 50 % des résidences présentent des valeurs manquantes 


df_dep_ratio = pd.merge(df_ratio_min, df , how="left", on=["departement", "departement"])


## II.1. Où se situent les Crous ? 

### II.1.A. Carte interactive des informations pratiques sur les Crous

Nous avons construit une carte interactive représentant l'ensemble des résidences Crous présentes en France métroplitaine et dans les départements et régions d'outre-mer. 
Cette première vue d'ensemble montre une concentration des résidences Crous dans les grandes métropoles françaises comme Paris, Lille, Lyon ou encore Bordeaux et dans les régions avec une forte population étudiante. 

La carte est interactive et permet de voir en un clic les principales informations de chaque résidence Crous (le nom de la résidence, son adresse, la surface moyenne des logements proposés et le loyer moyen.

In [ ]:
map = folium.Map(location=[48.7144587, 2.2453842], tiles="OpenStreetMap", zoom_start=11)

# Create a geometry list from the GeoDataFrame

geo_df_list = [[point.xy[1][0], point.xy[0][0]] for point in df.geometry] 


# Iterate through list and add a marker for each residence, color-coded by its type.

i = 0

for coordinates in geo_df_list:

    # Place the markers with the popup labels and data

    map.add_child(

        folium.Marker(

            location=coordinates,

            popup=

                 "Nom: " + str(df.title[i]) + "<br>"

                + "Moyenne Loyer: " + str(df["Mean Loyer"][i]) + "<br>"

                + "Moyenne surface: " + str(df["Mean Surface"][i]) + "<br>"

                + "Adresse: " + str(df.address[i]) + "<br>"

                + "Coordonnées: " + str(geo_df_list[i])

            ,

        

            icon=folium.Icon(color="blue"),

        )

    )

    i = i + 1

map

## II.2 Existent-ils des disparités entre les résidences Crous par département ? 

### II.2.A. Statistiques générales des résidences Crous

In [ ]:
df_stats = df[['Min Loyer', 'Max Loyer', 'Min Surface', 'Max Surface', 'Loyer_crous_prix_par_m2_surf_min', 'Loyer_crous_prix_par_m2_surf_max', 'Loyer_prive_m2']]
df_stats.describe()

Des statistiques descriptives ci-dessus, nous constatons que : 
+  La surface moyenne d'un studio en résidence Crous est de 17,5 m2 pour un loyer mensuel moyen de 316 euros.  
+  Pour une colocation ou un T1/grand studio, la surface moyenne est de 28 m2 pour un loyer mensuel moyen de 416 euros. 
+  L'écart-type des loyers entre les différentes résidences Crous est assez élevé ce qui implique des disparités significatives entre départements : pour un studio par exemple, l'écart-type du prix du loyer au m2 est proche de 6 euros pour une moyenne de 20 euros/m2. 

### II.2.B. Disparités en termes de surface entre les résidences Crous entre département 

Le loyer d'un logement dépend notamment de la surface de celui-ci, c'est pourquoi nous avons voulu comparer les éventuelles disparités entre département des résidences Crous sur la question de la superficie des logements proposés.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="whitegrid") 

# Initialize the matplotlib figure
f, ax = plt.subplots(figsize=(25, 25))

# Load the example car crash dataset
surfaces = df.dropna()

# Plot the total crashes
sns.set_color_codes("pastel")
sns.barplot(y="departement", x="Max Surface", data=surfaces,
            label="Surface grand studio/colocation", color="b")

# Plot the crashes where alcohol was involved
sns.set_color_codes("muted")
sns.barplot(y="departement", x="Min Surface", data=surfaces,
            label="Surface studio", color="b")

# Add a legend and informative axis label
ax.legend(ncol=2, loc="lower right", frameon=True, fontsize = 15)
ax.set(xlim=(0, 100), ylabel=" départements ",
       xlabel="Surfaces des logements Crous par département")
plt.title("Répartition des surfaces des logements dans les résidences Crous par département", size = 25) 


### II.2.C Disparité du prix du loyer au m2 entre les résidences Crous par département et comparaison avec le privé

Nous effectuons une comparaison du prix des loyers au m2 des résidences Crous par département. Nous utilisons un indicateur de prix au m2 pour prendre en considération les différences en termes de surface des logements proposés par les résidences Crous entre département et nous le comparons en même temps avec le prix du loyer par m2 proposé dans le parc privé. 

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
# Reorder it following the values of the first value:

ordered_df = df.dropna().groupby("departement").mean().sort_values(by='Loyer_crous_prix_par_m2_surf_max')
my_range=ordered_df.index
 
# The horizontal plot is made using the hline function
plt.hlines(y= my_range, xmin=ordered_df['Loyer_crous_prix_par_m2_surf_max'], xmax=ordered_df['Loyer_prive_m2'], color='black', alpha=0.4)
plt.scatter(ordered_df['Loyer_crous_prix_par_m2_surf_max'], my_range, color='darkcyan', alpha=1, marker = 'o', label='Loyer_crous_prix_par_m2_surf_max')
plt.scatter(ordered_df['Loyer_crous_prix_par_m2_surf_min'], my_range, color='firebrick', alpha=1,  marker = 's', label='Loyer_crous_prix_par_m2_surf_min')
plt.scatter(ordered_df['Loyer_prive_m2'], my_range, color='darkgreen', alpha=1 ,marker = '^', label='Loyer_prive_m2')
plt.legend()

    
# Add title and axis names
plt.yticks(my_range, ordered_df.index, fontsize= 10)
plt.title("Comparaison du loyer min/max en résidence Crous et avec le parc privé", loc='left')
plt.xlabel('Loyer', color = 'b')
plt.ylabel('Départements', color = 'b')
ax.legend(loc = 4, bbox_to_anchor = (1,0))
plt.style.use("ggplot")


# Show the graph
plt.show()

Même graphique en enlevant les départements avec un nombre trop important de valeurs manquantes 

Du graphique ci-dessous nous pouvons dire que : 
+ Le prix au m2 des loyers pour un studio en résidence Crous est quasiment toujours plus élevé que pour un logement de taille supérieur (généralement en colocation). Ce qui est assez cohérent, plus un logement est grand et loué en colocation moins il revient cher individuellement.
+ Les variations du prix du loyer au m2 dans les logements Crous est plus faible que dans le parc privé. 
+ Dans les grandes métropoles étudiantes, le prix au m2 du loyer en Crous est nettement inférieur à celui proposé dans le parc privé : à Paris par exemple, l'écart est d'environ 15 euros par m2. Les loyers du Crous n'augmentent donc pas même si les prix proposé dans le privé est sensiblement plus important. 
+ Dans les zones où la tension étudiante est plus faible, le prix du loyer au m2 dans le privé est souvent plus faible que celui proposé par les résidences Crous : cela s'explique principalement par le fait que dans ces départements, les logements proposés à la location sont souvent de plus grandes superficie (principalement T3), de ce fait ils ne sont pas très représentatifs des logements que rechercheraient un étudiant qui ne souhaite pas forcément être en colocation.

In [ ]:
fig, ax = plt.subplots(figsize=(20, 10))
# Reorder it following the values of the first value:

ordered_df = df_dep_ratio .dropna().groupby("departement").mean().sort_values(by='Loyer_crous_prix_par_m2_surf_max')
my_range=ordered_df.index
 
# The horizontal plot is made using the hline function
plt.hlines(y= my_range, xmin=ordered_df['Loyer_crous_prix_par_m2_surf_max'], xmax=ordered_df['Loyer_prive_m2'], color='black', alpha=0.4)
plt.scatter(ordered_df['Loyer_crous_prix_par_m2_surf_max'], my_range, color='darkcyan', alpha=1, marker = 'o', label='Loyer_crous_prix_par_m2_surf_max')
plt.scatter(ordered_df['Loyer_crous_prix_par_m2_surf_min'], my_range, color='firebrick', alpha=1,  marker = 's', label='Loyer_crous_prix_par_m2_surf_min')
plt.scatter(ordered_df['Loyer_prive_m2'], my_range, color='darkgreen', alpha=1 ,marker = '^', label='Loyer_prive_m2')
plt.legend()

    
# Add title and axis names
plt.yticks(my_range, ordered_df.index, fontsize= 10)
plt.title("Comparaison du loyer min/max en résidence Crous et avec le parc privé", loc='left')
plt.xlabel('Rent prices', color = 'b')
plt.ylabel('Départements', color = 'b')
ax.legend(loc = 4, bbox_to_anchor = (1,0))
plt.style.use("ggplot")


# Show the graph
plt.show()

## II.3 Répartition des étudiants et des résidences Crous 

Pour savoir dans quelle mesure les résidences Crous s'adaptent au nombre d'étudiant présent dans chaque département, nous avons comparé les dix départements avec le plus d'étudiants présents avec les dix départements avec le plus de résidences Crous. 

In [ ]:
nb_eleve_dep_top_10 = nb_eleve_dep_top_10.loc[(nb_eleve_dep_top_10['effectif_sans_cpge']>56740)] 
departments = nb_eleve_dep_top_10.index
Nb_élèves = nb_eleve_dep_top_10['effectif_sans_cpge']

fig, ax = plt.subplots(figsize=(20, 10)) #taille adéquate 

plt.bar(departments, Nb_élèves, color ='coral' )
plt.xticks(departments, rotation = 'vertical', size =10)
plt.title("Dix départements avec le plus d'étudiants", loc='left', size=20)
plt.xlabel('Départments', color = 'b') 
plt.ylabel("Départements avec le plus d'élèves", color = 'b')
plt.style.use("ggplot") #à utiliser pour améliorer le style 
plt.show()

Parmi les dix dépatements avec le plus de d'étudiants on retrouve 7 départements avec le plus de résidences Crous : il y a bien un effet d'accompagnement de la demande (étudiante) par les résidences Crous. 

In [ ]:


nb_res_dep_top_10 = nb_res_dep_top_10.loc[(nb_res_dep_top_10['index']>=23)] 
departments = nb_res_dep_top_10.index
Nb_res = nb_res_dep_top_10['index']

fig, ax = plt.subplots(figsize=(20, 10)) #taille adéquate 

plt.bar(departments, Nb_res, color ='plum')
plt.xticks(departments, rotation = 'vertical', size =10)
plt.title("Dix départements avec le plus de résidences Crous", loc='left', size=20)
plt.xlabel('Départments', color ='b') 
plt.ylabel("Départements avec le plus de résidences Crous", color ='b')
plt.style.use("ggplot")
plt.show()

### II.3.A. Répartition des résidences Crous en fonction du nombre d'étudiants par département

Enfin, nous avons voulu savoir si l'offre et la demande de logements en résidence Crous était homogène sur le plan national. Pour cela, nous avons réalisé une carte chloroplèthe qui permet d'illustrer les différences de pression de la demande étudiante de logements Crous entre les départements de France métroplitaine.  
+ **Lecture** : un département jaune signifie qu'il y a ratio assez faible entre le nombre de résidences et le nombre d'étudiants dans le département (entre 0 et 0.02%). 
+ **Interprétation** : La pression de demande de logements Crous est très marquée en Ile de France, en Bretagne et dans le Bas-Rhin où le pourcentage de résidence par étudiant est inférieur à 0,03 % quand ce ratio passe de 10 à 23 % dans plusieurs départements du sud de la France. 

In [ ]:
# Load the json file with departement coordinates
geoData = gpd.read_file("https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/france.geojson")
geoData.index = geoData.index.astype(str).astype(int)
fullData = geoData.merge(nb_residence_par_eleves,left_on=['code'], right_on=['code_insee'])
#Obtain dataframe with data + coordonate (more easy to manipulate) 

In [ ]:
"""
On récupère un fond de carte de la France.
"""

France = gpd.read_file("https://raw.githubusercontent.com/holtzy/The-Python-Graph-Gallery/master/static/data/france.geojson")

In [ ]:

fig, ax = plt.subplots(figsize=(20, 18))
France.plot(ax =ax, edgecolor='darkgrey', facecolor='lightgrey') #on affiche toute la France ainsi on n'a pas uniquement les départements ayant des Crous

# Set up the color sheme:
import mapclassify as mc
scheme = mc.Quantiles(nb_residence_par_eleves['Nb_by_students'], k=10) #on calcule 7 quantiles du nombre d'étudiants par résidences

# Map
maps = gplt.choropleth(fullData[(fullData["level_0"] != "Guyane") & (fullData["level_0"] != "La Réunion") &
                                        (fullData["level_0"] !="Martinique") & (fullData["level_0"] !="Guadeloupe")], 
    hue= "Nb_by_students" , 
    linewidth=.1,
    scheme=scheme, cmap='inferno_r',
    legend=True,
    edgecolor='black',
    ax=ax
                
);

ax.set_title('Nombre de résidences Crous par étudiants par département', fontsize=30)

**Conclusion** : 
+ Les résidences Crous tentent d'accompagner la demande étudiante en s'implantant au maximum dans les grandes aires étudiantes. 
+ Pour autant, la pression de la demande de logements sociaux dans les grandes métropoles françaises demeure toujours très importante. 
+ Néanmoins, et ce qui est plutôt rassurant c'est que le prix du loyer en résidence Crous est assez insensible au prix des loyers proposé dans le privé ainsi qu'à la tension de la demande étudiante : les logements Crous n'ont pas de visée lucrative et cherchent toujours à favoriser l'insertion dans la vie étudiante. 

# Partie III : Modélisation 
Dans un premier temps nous effectuons des régressions pour essayer de modéliser la variable Loyer. 

Ensuite, nous nous intéressons au prix par mètre carré des résidences Crous et essaierons de déterminer si ce-dernier est très sensible aux variations du prix par mètre carré des logements privés loués et de la demande des étudiants. En d'autres termes, nous essaierons de déterminer si le prix au mètre carré à la location des résidences crous suit la loi de marché.

## III.0 Préparation du dataframe
On créé un nouveau dataframe df2 qui correspond à df, duquel on retire les valeurs manquantes pour que les régressions fonctionnent.

In [ ]:
df2 = df
index_with_nan = df2.index[df2.isnull().any(axis=1)] #on regarde l'index des NaN sur toutes les colonnes du dataframe

df2.drop(index_with_nan,0, inplace=True) #on retire les lignes avec des valeurs manquantes sur le dataframe df2
df2 = df2.reset_index() #On réindexe df2 pour ne pas avoir des 'sauts' dans les indexes.


print(df.shape)
print(df2.shape)

## III.1 Premières régressions linéaires simples pour expliquer la variable Loyer

L'objectif de cette partie est de s'assurer de la cohérence de nos données. Nous effectuons pour cela, la régression du loyer maximal sur la surface maximale.

Dans la partie II.2.A. de statistiques descriptives de df, nous avons vu que les variables Max Loyer et Max Surface prenaient des valeurs respectivement entre 146 et 823 euros et entre 9 et 99 m². Il n'y a donc pas de valeurs incohérentes qui viendraient fausser les résultats.

On trace le nuage de points des loyers en fonction de la surface.

In [ ]:
plt.scatter(df2['Max Surface'], df2['Max Loyer'])
plt.title('Loyer en fonction de la surface en mètre carrés')
plt.xlabel('Surface (en mètre carré)')
plt.ylabel('Loyer (en euros)')
plt.show()

In [ ]:
from functions_for_modelling import * 
get_OLS_reg(df2['Max Surface'], df2['Max Loyer'])

*Analyse*:
- Le R² de 0.18 est plutôt faible, la variable explicative (ici la surface moyenne) n'est pas très utile pour prédire le résultat. 
- La p-value indique que l'on rejette H0 à 1%, le coefficient associé à la variable Max Surface est donc significatif.
- Le nombre d'observations est égal à 320, ce qui est assez faible. 
- La constante est assez élevée, égale à 320, ce qui peut s'expliquer par la nature de la variable expliquée, le loyer, qui prend des valeurs entre 146 et 818 euros.

- En admettant que la régression identifie un effet causal de la surface maximale, une hausse de 1m² de la surface entraînerait une hausse de 3 euros du loyer. Ce qui semble acceptable, partant d'un loyer de 320 euros. 

On trace ensuite la droite de régression linéaire:

In [ ]:
sns.regplot(x='Max Surface', y='Max Loyer', data=df2) 

In [ ]:
#tracer résidus en fonction de y^  model.predict()

On s'intéresse désormais à la possible hétéroscedasticité des résidus. 
On effectue le test de Breusch-Pagan afin d'avoir une indication plus précise sur l'homoscédasticité ou non des résidus.

In [ ]:
get_bp_test_OLS(df2['Max Surface'],df2['Max Loyer'])

_Analyse_:

- La p-value du test est inférieure à 0.05 donc le modèle semble hétéroscedastique.

- Le seul inconvénient du test de Breusch-Pagan est qu'il peut donner une faible p-value même quand le modèle n'est pas hétéroscedastique. En effet, si le modèle est imparfait, généralement en raison d'un biais de variable omise, le test peut alors indiquer une p-value faible.

On effectue une régression prenant en compte une possible hétéroscedasticité des résidus avec le module RLM (Robust Linear Method) de statsmodel. 

In [ ]:
get_RLM(df2['Max Surface'],df2['Max Loyer'])

_Analyse_:
- On remarque que l'écart type (std err) du coefficient associé à Max Surface a diminué. Il reste significatif puisqu'on rejette H0 à 1% (pvalue < 0.01).
- On remarque également que l'intervalle de confiance est plus restreint.

La régression est donc plus précise avec le module RLM. On privilegiera désormais le module RLM pour effectuer des régressions, puisque celui-ci prend en compte une possible hétéroscedasticité des résidus.

## III.2 Modélisation du prix du mètre carré des résidences Crous

### III. 2.A.  Le prix du mètre carré des appartements privés loués a-t-il une incidence sur le prix du mètre carré des résidences Crous ?

Pour répondre à cette question, nous effectuons la régression linéaire du prix du mètre carré des résidences Crous sur le prix au mètre carré des appartements privés loués.¶

In [ ]:
get_RLM(df2['Loyer_prive_m2'],df2['prix_par_m2_surf_max'])

_Analyse_:
- Le coefficient associé au prix du mètre carré des appartements privés loués est positif, il est de 0.1283 et est significatif au seuil de 1% (car la p-value est inférieure à 1%).

- Ainsi, en admettant que la régression identifie un effet causal du prix au mètre carré des logements privés loués, une hausse de 1 euro du prix du mètre carré des appartements privés loués entraîne une hausse de 12 centimes du prix du mètre carré des résidences crous. 
- Concrètement, si l'on considère un studio de 20m2 loué au sein d'une résidence Crous à Grenoble. Une hausse de 10 euros du prix au mètre carré des loyers de location privée à Grenoble conduit à une hausse de 1,4 euros du prix au mètre carré des loyers des résidences Crous de Grenoble. En particulier, le loyer de l'appartement considéré augmente donc de 28 euros par mois. (En admettant à nouveau que la régression identifie un effet causal du prix au mètre carré des loyers privés).

- La constante est de 14,3 ce qui paraît cohérent en réfléchissant en termes de prix au mètre carré. 

Les résidences Crous tendent donc à suivre les variations des prix de marché, mais faiblement.  

In [ ]:
sns.regplot(x='Loyer_prive_m2', y='prix_par_m2_surf_max', data=df2) 

### III. 2.B.  Le nombre d'étudiants scolarisés à proximité des résidences Crous ont-ils une incidence sur le prix au mètre carré de ces dernières ? 

Nous effectuons la régression du prix au mètre carré des logements Crous sur le nombre d'étudiants scolarisés à proximité.

On créé pour cela un nouveau dataframe.

In [ ]:
df2['Nbstudents'].describe()

La variable Nbstudents correspond au nombre d'étudiants scolarisés à proximité de résidences Crous.
On constate que le minimum de cette variable est de 0 ce qui signifie que certaines valeurs sont à 0 et vont fausser la régression.

On créée donc un nouveau dataframe duquel on retire ces valeurs nulles. Le nouveau dataframe, ne contient donc plus les résidences Crous associées à aucun étudiant. 

In [ ]:
i = df2[df2['Nbstudents']<=1000].index.values
df_students = df2.drop(i,axis=0)

In [ ]:
print(df_students['Nbstudents'].describe()) 
get_RLM(df_students['Nbstudents'],df_students['prix_par_m2_surf_max'])

_Analyse_:
- Le coefficient associé à la variable nombre d'étudiants est positif : il y a donc une association positive entre le nombre d'étudiants et le prix au mètre carré
- Le coefficient est significatif au seuil de 1% car la p-value = 0.004 <0.01.
- Le coefficient est très faible, proche de 0. Cependant, rapporté au nombre moyen d'étudiants qui est de l'ordre de la dizaine de millier, il ne semble pas incohérent. Il faut que l'augmentation du nombre d'étudiants soit d'un facteur 10^6 pour avoir un impact significatif. 
- Ainsi, en admettant que la régression identifie un effet causal du nombre d'étudiants, une hausse de 1000 élèves a un impact de 0.7 centimes sur le prix du mètre carré. Donc, une hausse de 100 000 élèves a un impact de près de 80 centimes sur le prix du mètre carré.

+ Interprétation : Ces résultats sont rassurants. Ils signifient que les résidences Crous ne spéculent pas sur la hausse de la demande et du nombre d'étudiants dans la région où ils se trouvent.

In [ ]:
sns.regplot(x='Nbstudents', y='prix_par_m2_surf_max', data=df2) 

### III. 2.C.  Y-a-t-il un effet commun du nombre d'étudiants scolarisés à proximité des résidences et du prix au mètre carré des loyers privés sur le prix au mètre carré des résidences Crous ? 

On effectue à présent une régression longue du prix au mètre carré dans les résidences Crous sur le nombre d'étudiants et le prix au mètre carré des appartements privés loués.

In [ ]:
get_RLM(df_students[['Nbstudents','Loyer_prive_m2']],df_students['prix_par_m2_surf_max'])

_Analyse_:
- Le coefficient associé à la variable Loyer_prive_m2 est significatif au seuil de 10% (p-value = 0.067 <10%) et positif. Ainsi, comme pour la régression simple (III.3.A), une hausse du prix au mètre carré des loyers des appartements privés loués d'une ville est associée à une hausse des loyers des résidences du Crous de cette même ville. Une augmentation de 1 euro du loyer privé conduit à une augmentation de 14 centimes du loyer crous, en supposant que la regression identifie un effet causal du prix au mètre carré des appatemenrs privés. 

- Il est étonnant que le coefficient associé au nombre d'étudiants soit négatif. Une baisse du nombre d'étudiants entraînerait une hausse du prix au mètre carré des résidences crous, ce qui ne coïncide pas avec la régression précédente (III.3.B) et avec les analyses obtenues dans la partie visualisation. *Les loyers des résidences Crous de Paris sont 100 euros plus chers que ceux des résidences de Dijon, ville qui compte moins d'étudiants que Paris.* Cependant, ce coefficient n'est pas significatif puisque sa p-value >60%.  

On arrive à la même conclusion que précédemment (III.3.A), les Crous suivent la loi du marché mais faiblement.

### III.2.D L'élasticité prix du mètre carré des résidences Crous 

On s'intérèsse enfin à l'élasticité prix du mètre carré des résidences Crous sur le nombre d'étudiant. 

Nous effectuons la régression linéaire simple du logarithme du prix par mètre carré dans les résidences Crous sur le logarithme du nombre d'étudiants pour obtenir l'élasticité prix du mètre carré des résidences Crous.

In [ ]:
x= np.log(df_students['Nbstudents'])
y= np.log(df_students['Loyer_prive_m2'])
get_RLM(x,y)

_Analyse_:
- Le coefficient associé au logarithme du nombre total d'étudiants est positif et significatif au seuil de 1%.

- Le coefficient correspond à l'élasticité prix du nombre d'étudiants, il peut être considéré comme l'élasticité prix de la demande, puisque les étudiants constituent l'essentiel des locataires des résidences Crous. 

- En supposant que la régression identifie un effet causal du nombre d'étudiants, une hausse de 100 points de pourcentage du nombre d'étudiants entraîne une hausse de 4 points de pourcentage du logarithme du prix au mètre carré des résidences Crous, toutes choses égales par ailleurs.

In [ ]:
df2['log_Nbstudents'] = np.log(df_students['Nbstudents'])
df2['log_Loyer_prive_m2'] = np.log(df_students['Loyer_prive_m2'])

In [ ]:
sns.regplot(x='log_Nbstudents', y='prix_par_m2_surf_max', data=df2) 

__Conclusion__:
Ces régressions 